In [11]:
import logging
logging.root.setLevel(logging.INFO)


In [12]:
import random

from xcs import XCSAlgorithm
from xcs.scenarios import Scenario

In [13]:
import random

from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

class HaystackProblem(Scenario):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None

    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
        
    # XCS Hosford42 functions
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.needle_value

    # XCS Pyalcs functions
    def step(self, action):
        done = not self.execute(action)
        sense_string = str(self.sense())
        raw_state = [s for s in sense_string]
        if not done:
            reward = 0
        else:
            reward = 1
        return raw_state, reward, done, _

In [14]:
logging.root.setLevel(logging.INFO)
scenario = HaystackProblem()

In [15]:
algorithm = XCSAlgorithm()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

In [16]:
model = algorithm.new_model(scenario)
model.run(scenario, learn=True)


In [17]:
print(model)
print(len(model))

0110#1#000#0#1#101100#101#0#100#000#00##01#00##10#100#111010##1001#1#001##111#1000#01#101#1#0101#1#01##0#11##0101001#10#011#####1##1#10100110#011000#0#00100#1#1010000#1##011010#1#1101#000###1#0000011110#01##11001#1#1##11##0111#00##1011#0001#01110#101000100##0#0#11101#111#0111010#100011##1110##0#01###1#00####00#11011001#110011#10101#1####0#0#100##11##01#0###100#1#11##00#00##0110#11#11#10010#0#0111##010011#100##000#0001##100001001###11111#1#111#0110#001#1##000#1#1##11#1##000000#110#0011010#00#01#1 => False
    Time Stamp: 29
    Average Reward: 0.0
    Error: 0.0
    Fitness: 0.15000850000000002
    Experience: 1
    Action Set Size: 1.0
    Numerosity: 1
#11#11#110##0###01001#1####01##010#011#0#10##1101#110#1#00#1101##01#00#01#0##110###0###0#1#10#1#000#10#1#1#010#001##111000##0#0###0#00#101#100#110#01#1###0000#1100110##00111#000#0#10100000#0#1##10###0#11#00#11100##1###10100001#11#0##1100#0#0#1##11#0111####0#11###1#00##1#100#00000100#0101#110#0##0#0010#001#1#101#0#10##0#000#000110100#10#

In [18]:
for rule in model:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)

In [19]:
from lcs.agents.xcs import XCS, Configuration

cfg = Configuration(number_of_actions=4,
                    gamma=0,
                    metrics_trial_frequency=5,
                    covering_wildcard_chance=0.99
                    )


In [20]:
agent = XCS(cfg)
explore_population, explore_metrics = agent.explore(scenario, 1000, False)

AssertionError: 

In [ ]:
print(len(explore_population))
for rule in explore_population:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)